# Load teslafi to MySQL using PySpark

In [ ]:
!echo $VIRTUAL_ENV
!env|grep SPARK
!echo $SPARK_HOME
# !pip install pandas findspark

In [ ]:
import pyspark
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, IntegerType
import pandas as pd
from configparser import ConfigParser

import os
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /usr/share/java/mariadb-java-client.jar pyspark-shell'

import findspark
findspark.init()


In [ ]:
# !env|grep SPARK
# !cp -v /usr/share/java/mariadb-java-client.jar $SPARK_HOME/jars/

In [ ]:
# spark = SparkSession.builder.config("spark.jars", "/usr/share/java/mariadb-java-client.jar").appName("TeslafiLoad").getOrCreate()
spark = SparkSession.builder.appName("TeslafiLoad").getOrCreate()
sc = spark.sparkContext


In [ ]:
# os.listdir(path='/data/data-files/teslafi/')

In [ ]:
# File location and type
csvdir = "/data/data-files/teslafi"
file_type = "csv"
# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# for path,name in [(f.path,f.name) for f in dbutils.fs.ls(csvdir) if f.path.endswith('.csv')  ]:
for fname in os.listdir(path=csvdir):
    if fname.endswith('.csv'):
        print(fname)
        df = spark.read.format('csv') \
        .option("inferSchema", infer_schema) \
        .option("header", first_row_is_header) \
        .option("sep", delimiter) \
        .load(csvdir + '/' + fname)
        infer_schema = "false" # reset inferrence after first file is read
        df.write.format('jdbc').options(
            url='jdbc:mysql://172.16.17.18:30306/teslafi',
            driver='org.mariadb.jdbc.Driver',
            dbtable='teslafi',
            user='gregj',
            password='dbuser').mode('append').save()

In [ ]:
df.dtypes

In [ ]:
!env | grep SPARK
